In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import wandb
import copy
import time
from datetime import datetime
from pytz import timezone
import seaborn as sns

import cv2

from torchvision.transforms import ToTensor,Compose,ColorJitter,RandomRotation,RandomHorizontalFlip,Resize,CenterCrop,Normalize,GaussianBlur,RandomCrop

import timm

import torch
import torch.nn as nn
import torch.optim as optm
import torch.nn.functional as F
import torch.distributions as dist
from torch.utils.data import Dataset, DataLoader

import torchvision

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import gridspec

from torchinfo import summary
from tqdm.auto import tqdm

%matplotlib inline
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cwd=os.path.dirname(os.getcwd())
KST=timezone('Asia/Seoul')
date=str(datetime.now().astimezone(KST))[:10]

In [2]:
#프로젝트 이름 설정과 저장경로
project_name='ageefficientnet_with_label_smoothing'

In [3]:
#Randomness 제어
random_seed=2021
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [4]:
transform={'trans':Compose([
    Resize((256*512//384,256)),
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
]),'regtrans':Compose([
    Resize((256*512//384,256)),
    GaussianBlur(3),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    RandomCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
]),'uptrans':Compose([
    Resize((256*512//384,256)),
    RandomHorizontalFlip(),
    RandomRotation(20),
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])}

def make_images(meta,img_dir,train):
    images=[]
    labels=[]
    ages=[]
    data=pd.DataFrame({'images':[],'labels':[]})
    if train:
        for idx in range(len(meta)):
            folder_path=os.path.join(img_dir, meta.path.iloc[idx])
            for img in os.listdir(folder_path):
                if '._' in img:
                    continue
                images.append(os.path.join(folder_path,img))
                labels.append((('incorrect' in img)+('normal' in img)*2)*6+(meta.gender.iloc[idx]=='female')*3+(30<=meta.age.iloc[idx])+(60<=meta.age.iloc[idx]))
                ages.append(meta.age.iloc[idx])
    else:
        for img_id in meta.ImageID:
            images.append(os.path.join(img_dir, img_id))
        labels=[0 for _ in range(len(images))]
    data['images']=images
    data['labels']=labels
    data['ages']=ages
    return data

class ImageDataset(Dataset):
    def __init__(self,transform=transform,mod='train'):
        self.mod=mod
        self.train=mod!='test'
        self.md=['info','train']
        self.path=[os.path.join(cwd,'input/data/eval'),os.path.join(cwd,'input/data/train')]
        self.meta=pd.read_csv(os.path.join(self.path[self.train], f'{self.md[self.train]}.csv'))
        self.img_dir=os.path.join(self.path[self.train],'images')
        self.classes=[('Wear','Incorrect','Not Wear'),('남','여'),('<30','>=30 and <60','>=60')]
        self.trans=transform
        self.criteria=1*(self.meta['age']>=30)+1*(self.meta['age']>=60)+3*(self.meta['gender']=='female')
        if self.train:
            self.train_data,self.val_data=train_test_split(self.meta,test_size=0.2,stratify=self.criteria)
            self.train_data=make_images(self.train_data,self.img_dir,self.train)
            self.val_data=make_images(self.val_data,self.img_dir,self.train)
    
    def __len__(self):
        return len(self.train_data) if self.mod=='train' else len(self.val_data)
        
    
    def __getitem__(self, idx):
        if self.mod=='valid':
            images=self.val_data['images']
            labels=self.val_data['labels']
            ages=self.val_data['ages']
            age=ages.iloc[idx]
        elif self.mod=='train':
            images=self.train_data['images']
            labels=self.train_data['labels']
            ages=self.train_data['ages']
            age=ages.iloc[idx]

        label=labels.iloc[idx]
        image=Image.open(images.iloc[idx])
        
        if self.mod=='train':
            if True:
                image=self.trans['uptrans'](image)
                return image,label,age
            else:
                image=self.trans['regtrans'](image)
                return image,label,age
        else:
            image=self.trans['trans'](image)
            return image,label,age
        return image,label

def conf_mat(y_true,y_pred):
    cm=confusion_matrix(y_true,y_pred)
    norm_cm=cm/np.sum(cm, axis=1)[:,None]
    indices=['wear,m,<30','wear,m,mask<>','wear,m,mask60','wear,f,<30','wear,f,<>','wear,f,60','inc,m,<30','inc,m,<>','inc,m,60','inc,f,<30','inc,f,<>','inc,f,60','nom,m,<30','nom,m,<>','nom,m,60','nom,f,<30','nom,f,<>','nom,f,60']
    cm=pd.DataFrame(norm_cm,index=indices,columns=indices)
    fig=plt.figure(figsize=(11,9))
    sns.heatmap(cm,annot=True)
    return fig


class SkewNormal(dist.normal.Normal):
    def __init__(self,mean=0,std=1,skew=0):
        super().__init__(loc=mean,scale=std)
        self.skew=skew
    def skewcdf(self,x):
        return 0.5 * (1 + torch.erf(self.skew*(x - self.loc) * self.scale.reciprocal() / np.sqrt(2)))

    def log_p(self,x):
        return self.log_prob(x)+torch.log(self.skewcdf(x))-torch.log(torch.tensor(2.))

def label_smoothing(label):
    label-=18
    label_0=SkewNormal(11.,7,-20).log_p
    label_11=SkewNormal(12.,7,20).log_p
    label_12=SkewNormal(41,7,-20).log_p
    label_2=dist.Normal(43.7,.65).log_prob
    return torch.exp(torch.vstack((label_0(label),torch.log(torch.exp(label_11(label))+torch.exp(label_12(label))),label_2(label))).T)

In [5]:
label_smoothing(torch.tensor([18,29,30,59,60]))

tensor([[8.2901e-03, 1.0123e-09, 0.0000e+00],
        [1.4248e-02, 6.3214e-05, 0.0000e+00],
        [6.0288e-05, 1.4253e-02, 0.0000e+00],
        [0.0000e+00, 1.4253e-02, 1.0996e-04],
        [0.0000e+00, 6.3214e-05, 2.0075e-02]])

# 부분 모델 학습과 모델저장

In [6]:
#사용할 부분 모델 불러오기
net_mask = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=3)
net_gender = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=2)
net_age = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=3)

In [7]:
#학습 설정하기
num_epochs=1000
batch_size=60
learning_rate=0.0001
models={'mask':net_mask,'gender':net_gender,'age':net_age}
weights={'age': torch.tensor([900/1281*10.8,900/1227*11,900/192]).to(device),'mask':torch.tensor([7/15,7/3,7/3]).to(device),'gender':torch.tensor([2700/(2*1042),2700/(2*1658)]).to(device)}


In [ ]:
#실행 파일 저장 경로

        
#부분 모델 사전 학습
for key in ['age']:
    
    now='Runned'+date+'_'.join(str(datetime.now().astimezone(KST))[10:16].split(':'))
    base_dir=os.path.join('.',f'saved/{project_name}/{now}_{key}')
    wandb_dir=base_dir
    checkpoint_dir=''
    model_dir=''
    result_dir=''
    dirs={'model':model_dir,'checkpoint':checkpoint_dir,'result':result_dir}
    for pth in ['model','checkpoint','result']:
        dirs[pth]=os.path.join(base_dir,pth)
        if not os.path.isdir(dirs[pth]):
            os.makedirs(dirs[pth])
    
    weight=weights[key]
    config={'epochs':num_epochs,'transform':'uptrans','batch_size':batch_size,'learning_rate':learning_rate,'gamma':0.75,'weight':weight,'f1_avg':'weighted','norm1':[11,7,-20],'norm2':[(12,41.7),7,20],'norm3':[43.7,.65]}
    data=ImageDataset()
    wandb.init(project=project_name+'_'+key,config=config,dir=wandb_dir)
    print('training:',key)
    model=models[key]
    model.to(device)
    optim=optm.Adam(model.parameters())
    scheduler = optm.lr_scheduler.StepLR(optim, step_size=1, gamma=config['gamma'])
    if key!='age':
        loss_fn=nn.CrossEntropyLoss(weight=weight)
    else:
        loss_fn=nn.BCEWithLogitsLoss(pos_weight=weight)
    best_f1=0.
    best_epoch=0
    stopper=0
    for e in range(num_epochs):   
        for mod in ['train','valid']:
            print('mod:',mod)
            data.mod=mod

            if mod=='train':
                dataloader=DataLoader(data,batch_size=batch_size,shuffle=True,pin_memory=True,num_workers=2)
                model.train()
            else:
                dataloader=DataLoader(data,shuffle=False)
                model.eval()

            with tqdm(dataloader) as pbar:
                running_loss=0.
                running_acc=0.
                prev_f1=0.
                tot_pred=torch.tensor([]).to(device)
                tot_label=torch.tensor([]).to(device)
                cm_pred=torch.tensor([]).to(device)
                cm_label=torch.tensor([]).to(device)
                
                for n,(image,label,age) in enumerate(pbar):
                    image=image.to(device)
                    label=label.to(device)
                    age=age.to(device)
                    classes={'mask':label//6,'gender':(label//3)%2,'age':label%3}
                    class_ind={'age':(1,6*classes['mask']+3*classes['gender']),'gender':(3,6*classes['mask']+classes['age']),'mask':(6,classes['age']+3*classes['gender'])}
                    cls_label=classes[key]
                    
                    
                    logit=model(image)
                    _,pred=torch.max(logit,1)
                    pred=class_ind[key][0]*pred+class_ind[key][1]
                    if data.train:
                        if key=='age':
                            loss=loss_fn(logit,label_smoothing(age))
                        else:
                            loss=loss_fn(logit,cls_label)
                        running_loss+=loss.item()*image.size(0)
                        running_acc+=torch.sum(pred==label)/image.size(0)
                        
                    if mod=='train':
                        optim.zero_grad()
                        loss.backward()
                        optim.step()
                        
                    tot_pred=torch.hstack((tot_pred,pred))
                    tot_label=torch.hstack((tot_label,label))
                    cm_pred=torch.hstack((cm_pred,pred))
                    cm_label=torch.hstack((cm_label,label))
                    if data.train:
                        pbar.set_postfix({'epoch' : e, 'loss' : running_loss/(n+1), 'accuracy' : float(running_acc)/(n+1),'F1 score':f1_score(tot_label.cpu(),tot_pred.cpu(),average='weighted')})
                        
                            
                if data.train:
                    epoch_loss=running_loss/len(dataloader.dataset)
                    epoch_acc=running_acc*image.size(0)/len(dataloader.dataset)
                    epoch_f1_w=f1_score(tot_label.cpu(),tot_pred.cpu(),average='weighted')
                    epoch_f1_m=f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')
                    print(f"현재 epoch-{e}의 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, F1 score macro: {epoch_f1_m:.3f}, F1 score weight: {epoch_f1_m:.3f}")
                    if mod=='train':
                        scheduler.step()
                        wandb.log({'training loss' : epoch_loss, 'training accuracy':epoch_acc,'training f1 score macro':epoch_f1_m,'training f1 score':epoch_f1_w})
                    
        if mod=='valid':
            cm=conf_mat(cm_label.cpu(),cm_pred.cpu())
            cm.savefig(f'./saved/{project_name}/{now}_{key}/checkpoint/checkpoint_{key}_2{str(e).zfill(3)}.jpg')
            wandb.log({f'{str(e).zfill(3)}_confusion_matrix':wandb.Image(cm),'loss' : epoch_loss, 'accuracy':epoch_acc,'f1 score macro':epoch_f1_m,'f1 score':epoch_f1_w})
            plt.close()
            if (prev_f1<epoch_f1_w) and not (best_f1<epoch_f1_w):
                torch.save({'epoch':e,'loss':loss,'model_state_dict':model.state_dict(),'optimizer_state_dict':optim.state_dict()},f"{dirs['checkpoint']}/{'2'+str(e).zfill(3)}_checkpoint_{key}_{epoch_f1_w:.5f}_{epoch_acc:.3f}_{epoch_loss:.3f}.pt")
            if best_f1<epoch_f1_w:
                stopper=0
                best_epoch=e
                best_f1=epoch_f1_w
                torch.save({'epoch':e,'loss':loss,'model_state_dict':model.state_dict(),'optimizer_state_dict':optim.state_dict()},f"{dirs['checkpoint']}/{'2'+str(e).zfill(3)}_checkpoint_{key}_best_{epoch_f1_w:.5f}_{epoch_loss:.3f}_{epoch_acc:.3f}.pt")
            elif stopper==5:
                break
            else:
                stopper+=1
    print(best_epoch)
    wandb.finish()

wandb: Currently logged in as: tentoto (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



training: age
mod: train



현재 epoch-0의 평균 Loss : 0.161, 평균 Accuracy : 0.871, F1 score macro: 0.703, F1 score weight: 0.703
mod: valid



현재 epoch-0의 평균 Loss : 0.145, 평균 Accuracy : 0.893, F1 score macro: 0.724, F1 score weight: 0.724
mod: train
